Imports

In [2]:
import bs4 as bs
import cv2
import csv
import glob
import pytesseract
import os
import urllib.request
import httplib2
from inaFaceAnalyzer.inaFaceAnalyzer import ImageAnalyzer

from PIL import Image
from sentence_transformers import SentenceTransformer, util
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification


2025-02-26 15:40:34.198447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-26 15:40:34.198484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-26 15:40:34.199766: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 15:40:34.207679: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-26 15:40:35.313542: W tensorflow/compiler/tf2

get_gender function 
input: list of video frame locations
output: list with 0 for M, 1 for F, 2 for neither, 3 for both

In [ ]:
inaFaceAnalyzer = ImageAnalyzer()

video_location = "test_videos/"

def get_gender(frame_list):
    frame_list=filter(lambda x: x.endswith(".jpg"),frame_list)
    frame_list=list(map(lambda x: video_location+x,frame_list))
    df = inaFaceAnalyzer(frame_list)
    df_2 = df.groupby("frame")['sex_label'].apply(list)
    ret = []
    for row in df_2:
        if len(row)>1:
            if len(set(row))>1: ret.append(3)
            else: 
                if row[0]=='f':
                    ret.append(1)
                else:
                    ret.append(0)
        else:
            if row[0]=='f': ret.append(1)
            else: ret.append(0)
    for frame_name in frame_list:
        if frame_name not in df['frame'].tolist():
            ret.append(2)
    
    return ret

2025-02-26 15:40:40.634833296 [E:onnxruntime:Default, provider_bridge_ort.cc:1862 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1539 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcudnn_adv.so.9: cannot open shared object file: No such file or directory

2025-02-26 15:40:40.634849907 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:993 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Require cuDNN 9.* and CUDA 12.*. Please install all dependencies as mentioned in the GPU requirements page (https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements), make sure they're in the PATH, and that your GPU is supported.


Test

In [6]:
ret=get_gender(["mcdoindeed2.jpg"])
print("M identified "+str(ret.count(0)))
print("F identified "+str(ret.count(1)))
print("Neither identified "+str(ret.count(2)))
print("Both identified "+str(ret.count(3)))


M identified 1
F identified 0
Neither identified 0
Both identified 0
